<a href="https://colab.research.google.com/github/adhello007/PersonalProjects-DeepLearning/blob/main/RoadSignLocalizationV1_MemoryError.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
import os
os.listdir('/root')

['.bashrc',
 '.profile',
 '.jupyter',
 '.local',
 '.tmux.conf',
 '.cache',
 '.ipython',
 '.config',
 '.npm',
 '.wget-hsts',
 '.keras',
 '.launchpadlib']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir ~/.kaggle #create a directory


In [ ]:
!cp '/content/drive/MyDrive/kaggle.json' ~/.kaggle/ #copy the json file to this directory

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json #giving permission to read and write

In [ ]:
! kaggle datasets download andrewmvd/road-sign-detection

In [ ]:
import os
path = '/root/.kaggle'
os.listdir(path)

In [ ]:
!unzip road-sign-detection.zip


In [ ]:
import cv2
import tensorflow as tf
import numpy as np
import pandas as pd
import xml.etree.ElementTree as xt
from bs4 import BeautifulSoup
from glob import glob

In [ ]:
xml_eg = '/content/annotations/road0.xml'

In [ ]:

with open(xml_eg,'r') as f:
  data = f.read()
  content = BeautifulSoup(data,'xml')

content

In [ ]:
path = sorted(glob('/content/annotations/*.xml')) #this is the annotations path
path

In [ ]:
images_path = sorted(glob('/content/images/*.png')) #this is the images path
images_path

The Code below basically compares if the annotation paths and the images path are arranged in the same order in their respective lists.

In [ ]:
#wrote this to make sure every image has its corresponding bbox values in the correct values
num = len(images_path)
a=0
for i in range(num):
  pat = path[i]
  img = images_path[i]
  filename1 = pat.split('/')[-1] #got the last part of path
  filename2 = img.split('/')[-1]

  key1 = filename1.split('.')[0] #got the 0th index sep by .
  key2 = filename2.split('.')[0]
  if(key1!=key2):
    print("not the same")
    a+=1

In [ ]:
a #this is the number of annotations and imgs which do not match

In [ ]:
#We're extracting every bbox value and also extracting the labels into a list y
path = sorted(glob('/content/annotations/*.xml'))
y = []

for file in path:
  label =0
  info  = xt.parse(file)
  root = info.getroot()
  obj = root.find('object')

  label_obj  = obj.find('name').text
  if "trafficlight" in label_obj:
    label = 1
  elif "stop" in label_obj:
    label = 2
  elif "speedlimit" in label_obj:
    label = 3
  elif "crosswalks" in label_obj:
    label = 4

  bndbox = obj.find('bndbox')
  xmin = (bndbox.find('xmin').text)
  xmax = (bndbox.find('xmax').text)
  ymin = (bndbox.find('ymin').text)
  ymax = (bndbox.find('ymax').text)

  y.append([int(label),int(xmin),int(ymin),int(xmax),int(ymax)])

In [ ]:
len(y)

In [ ]:
y[513]

In [ ]:
x = []

images_path = sorted(glob('/content/images/*.png'))

for image in images_path:
  img = cv2.imread(image) #this img is a numpy array
  x.append(img) #since x is a list , its a lotta numpy arrays in the list


In [ ]:
type(x[0])

In [ ]:
x[0].shape

In [ ]:
from PIL import Image
image = Image.open(images_path[0]) #you can open the image like this

In [ ]:
type(image)

In [ ]:
image.size #width, height

Now that we have the images and the targets:containing the class and its bboxes, we have to resize the images and their respective bounding boxes.

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x[0])


In [ ]:
import plotly.express as px
bbox = y[513][12:]
img_eg = x[513]
fig = px.imshow(img_eg)
fig.add_shape(type='rect', x0=bbox[0],x1=bbox[2],y0=bbox[1],y1=bbox[3],xref='x',yref='y',line_color='cyan')
print(bbox)

In [ ]:
y[513][0] #its class/label

In [ ]:
#lets make a new directory for resized images

os.mkdir('resized_imgs/')

In [ ]:
#playing around with lists
bblist = []
for subset in y:
  bbvalues = subset[1:5]
  bblist.append(bbvalues)

bblist

In [ ]:
eg = '/content/images/road1.png'
img = cv2.imread(eg) #opencv opens images in BGR format and not RGB. Here it doesnt really need to be converted to an rgb image.
plt.imshow(img)

In [ ]:
img = Image.open(eg) #opening the same image with Image.open() and showing its resized value.
img2 = img.resize((800,800))
plt.imshow(img2)

Here we are resizing the images one by one, saving the resized image, resizing their bboxes and saving them in a resized_bblist[]

In [ ]:

width = 400
height = 400
output_folder = '/content/resized_imgs'
resized_bblist =  []
i =0
for image in images_path:
  img = Image.open(image)
  original_width, original_height = img.size

  ratio_width = width / original_width
  ratio_height = height / original_height
  #resized the image
  resized_image = img.resize((width,height))

  #save the image
  filename = os.path.basename(image)
  output_path = os.path.join(output_folder,filename)
  resized_image.save(output_path)

  #that images' bbox
  xmin = bblist[i][0]
  ymin = bblist[i][1]
  xmax = bblist[i][2]
  ymax = bblist[i][3]
  i+=1
  nxmin = int(xmin * ratio_width)
  nymin = int(ymin * ratio_height)
  nxmax = int(xmax * ratio_width)
  nymax = int(ymax * ratio_height)
  resized_bblist.append([int(nxmin), int(nymin), int(nxmax), int(nymax)])


In [ ]:
resized_bblist[0]

In [ ]:
#extracting the resized_images and reading them in using opencv
resized_images = []
resized_path = sorted(glob('/content/resized_imgs/*.png'))

for image in resized_path:
  img = cv2.imread(image)
  resized_images.append(img)

In [ ]:
type(resized_images[0])

In [ ]:
#viewing the resized image and its bbox
bbox = resized_bblist[210]
img = resized_images[210]
fig = px.imshow(img)
fig.add_shape(type='rect',x0=bbox[0],x1=bbox[2],y0=bbox[1],y1=bbox[3],xref='x',yref='y',line_color='cyan')

In [ ]:
#creating a function to take the class label from y and joining it with the new resized bbox values to merge into a new list.
def merge_lists(list1,list2):
  merged_list = [[l1[0]] +l2 for l1,l2, in zip(list1,list2)]
  return merged_list

In [ ]:
merged_list = merge_lists(y,resized_bblist)

In [ ]:
merged_list[210]

In [ ]:
len(merged_list)

In [ ]:
#convert the resized images and the bboxes to a numpy array
resized_images = np.array(resized_images)
merged_lab_bbox = np.array(merged_list) #this has labels and bbox

In [ ]:
#shuffle the clean dataset
resized_images = np.random.permutation(resized_images)
merged_lab_bbox = np.random.permutation(merged_list)

In [ ]:
merged_lab_bbox[0]

In [ ]:
resized_images[2].shape , merged_lab_bbox[2].shape

In [ ]:
#IMPORTANT, we dont have an X_test cause we can test on a new image. we really need val dataset.
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(resized_images,merged_lab_bbox,test_size=0.2)

In [ ]:
type(X_train)

In [ ]:
X_train.shape

In [ ]:
X_val.shape #validation set

In [ ]:
y_train[0]

In [ ]:
#normalize the Xtrain and Xval values
X_train = X_train/255.
X_val = X_val/255.

Lets Start Training the Model

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torchvision.transforms as transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
num_classes = 5 #4 + 1 background

in_features = model.roi_heads.box_predictor.cls_score.in_features

#replace the pretrained head with a new one to perform localization and classification
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)


By subclassing torch.utils.data.Dataset, you can create a dataset object that can be used with PyTorch's data loading utilities, such as torch.utils.data.DataLoader, to efficiently load and process data during training or inference. A custom dataset typically represents a collection of data samples and their corresponding labels or annotations.

In [ ]:
#convert numpy images to a pytorch tensor

import torch
import torchvision.transforms as transforms

num_train_images = X_train.shape[0]
num_val_images = X_val.shape[0]

transform = transforms.Compose([
    transforms.ToTensor()
])

tensor_train_images = []
tensor_val_images = []
for i in range(num_train_images):
  image = X_train[i]
  preproc_img = transform(image)
  tensor_train_images.append(preproc_img)

tensor_train_images = torch.stack(tensor_train_images)

#Now for val images
for i in range(num_val_images):
  image = X_val[i]
  preproc_img = transform(image)
  tensor_val_images.append(preproc_img)

tensor_val_images = torch.stack(tensor_val_images)

In [ ]:
tensor_train_images.shape ,  tensor_val_images.shape

In [ ]:
#now write a function to convert a bounding box to a pytorch tensor
def convert_bbox_to_tensor(bbox):
  x1,y1,x2,y2 = bbox
  bbox_tensor = torch.tensor([x1,y1,x2,y2], dtype=torch.float32)
  return bbox_tensor

In [ ]:
#nowe use the function created above to create a pytorch bbox.
tensor_train_bboxes = []
tensor_val_bboxes = []
#for train bboxes
for i in range(num_train_images):
    bbox = y_train[i][1:]  # Exclude the label from the bbox values

    preproc_bbox = convert_bbox_to_tensor(bbox)  # Convert bbox numpy to a PyTorch tensor
    tensor_train_bboxes.append(preproc_bbox)

tensor_train_bboxes = torch.stack(tensor_train_bboxes)


#now for validation bboxes
for i in range(num_val_images):
    bbox = y_val[i][1:]  # Exclude the label from the bbox values

    preproc_bbox = convert_bbox_to_tensor(bbox)  # Convert bbox numpy to a PyTorch tensor
    tensor_val_bboxes.append(preproc_bbox)

tensor_val_bboxes = torch.stack(tensor_val_bboxes)

In [ ]:
# write a function to convert labels into pytorch tensors
def convert_labels_to_tensor(label):
  label_tensor = torch.tensor([label], dtype=torch.float32)
  return label_tensor

In [ ]:
tensor_train_labels = []
tensor_val_labels = []
for i in range(num_train_images):
  label = y_train[i][0]

  labeled = convert_labels_to_tensor(label)
  tensor_train_labels.append(labeled)

tensor_train_labels = torch.stack(tensor_train_labels)


#now for val labels
for i in range(num_val_images):
  label = y_val[i][0]

  labeled = convert_labels_to_tensor(label)
  tensor_val_labels.append(labeled)

tensor_val_labels = torch.stack(tensor_val_labels)

Now create the Main Road Sign Dataset FUnciton

In [ ]:
from torch.utils.data import Dataset
class RoadSignDataset(Dataset):
    def __init__(self, images, bounding_boxes, labels):
        self.images = images
        self.bounding_boxes = bounding_boxes
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        bounding_box = self.bounding_boxes[index]
        label = self.labels[index]

        return image, {'boxes': bounding_box, 'labels': label}


In [ ]:
#load all of this in data loader
from torch.utils.data import Dataset, DataLoader
train_dataset = RoadSignDataset(tensor_train_images, tensor_train_bboxes, tensor_train_labels)
val_dataset = RoadSignDataset(tensor_val_images, tensor_val_bboxes, tensor_val_labels)
train_dl = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size = 16, shuffle= True)

In [ ]:
train_dl , val_dl

We will use a very simple pre-trained resNet-34 model as our model. Since we have two tasks here, there are two final layers - the bounding box regression and the image classifier

In [ ]:
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F
class BB_model(nn.Module):
    def __init__(self):
        super(BB_model, self).__init__()
        resnet = models.resnet34(pretrained=True)
        layers = list(resnet.children())[:8]
        self.features = nn.Sequential(*layers)
        self.classifier = nn.Sequential(nn.BatchNorm1d(512), nn.Linear(512, 4))
        self.bb = nn.Sequential(nn.BatchNorm1d(512), nn.Linear(512, 4))

    def forward(self, x):
        x = self.features(x)
        x = F.relu(x)
        x = nn.AdaptiveAvgPool2d((1,1))(x)
        x = x.view(x.shape[0], -1)
        return self.classifier(x), self.bb(x)

In [ ]:
resnet = models.resnet34(pretrained=True)
list(resnet.children())[:8]

To calculate the loss, we need to take into account both the classification loss and the bounding box regression loss, so we use a combination of cross entropy and L1 loss

In [ ]:
model = BB_model()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.006)
epochs = 30

In [ ]:
import torch.nn.functional as F

def calculate_loss(outputs, targets, lambda_coord=5.0):
    pred_class, pred_bb = outputs
    true_class, true_bb = targets['labels'], targets['boxes']


    if 'labels' not in targets or 'boxes' not in targets:
        raise ValueError("Targets dictionary should contain 'labels' and 'boxes'.")


    classification_loss = F.cross_entropy(pred_class, true_class.squeeze().long()) # Classification loss using cross-entropy


    localization_loss = F.smooth_l1_loss(pred_bb, true_bb, reduction='mean')


    total_loss = classification_loss + lambda_coord * localization_loss  # Total loss as a combination of class and localization los

    return total_loss


In [ ]:
def train():
    for i in range(epochs):
        model.train()
        total = 0
        sum_loss = 0


        for images, targets in train_dl:
            images = images.cuda().float()
            bounding_boxes = targets['boxes'].cuda().float()
            labels = targets['labels'].cuda()

            # Forward paaass and loss calc
            outputs = model(images)
            loss = calculate_loss(outputs, {'boxes': bounding_boxes, 'labels': labels})

            # backprop and opt
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total += images.size(0)
            sum_loss += loss.item()

        train_loss = sum_loss / total

        # Validatn loop
        model.eval()
        val_total = 0
        val_sum_loss = 0
        val_correct = 0

        for val_images, val_targets in val_dl:
            val_images = val_images.cuda().float()
            val_bounding_boxes = val_targets['boxes'].cuda().float()
            val_labels = val_targets['labels'].cuda()

            # Forward p and loss calcu or validation
            val_outputs = model(val_images)
            val_loss = calculate_loss(val_outputs, {'boxes': val_bounding_boxes, 'labels': val_labels})

            # Calculate val accuracy
            val_preds = torch.argmax(val_outputs[1], dim=1)  # Assuming label predictions are in index 1
            val_correct += torch.sum(val_preds == val_labels.squeeze()).item()

            val_total += val_images.size(0)
            val_sum_loss += val_loss.item()

        val_loss = val_sum_loss / val_total
        val_acc = val_correct / val_total

        print(f"Epoch [{i+1}/{epochs}]. train_loss {train_loss:.3f} val_loss {val_loss:.3f} val_acc {val_acc:.3f}")
        val_loss = val_sum_loss.item()


In [ ]:
train()